In [2]:
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
import fiona
import os
import numpy as np

In [11]:
pasta_imagens = 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/0_raw/OPTICAL/LT05_L2SP_227062_20050701_20200902_02_T1'
arquivo_vetor = 'G:/Meu Drive/INPE/projeto_dissertacao/3_gis/projetos_GIS/PALSAR_2008_SDNLM_amplitude_extention.shp'
pasta_destino = pasta_imagens
dst_crs = 'EPSG:31981'

In [12]:
sufixos= ['B1.TIF', 'B2.TIF', 'B3.TIF', 'B4.TIF', 'B5.TIF', 'B7.TIF']

In [22]:
# Lista todos os arquivos de imagem na pasta
arquivos_imagens = [os.path.join(pasta_imagens, arquivo) for arquivo in os.listdir(pasta_imagens) 
                    if any(arquivo.endswith(sufixo) for sufixo in sufixos)]

arquivos_imagens 

['G:/Meu Drive/INPE/projeto_dissertacao/0_images/0_raw/OPTICAL/LT05_L2SP_227062_20050701_20200902_02_T1\\LT05_L2SP_227062_20050701_20200902_02_T1_SR_B1.TIF',
 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/0_raw/OPTICAL/LT05_L2SP_227062_20050701_20200902_02_T1\\LT05_L2SP_227062_20050701_20200902_02_T1_SR_B2.TIF',
 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/0_raw/OPTICAL/LT05_L2SP_227062_20050701_20200902_02_T1\\LT05_L2SP_227062_20050701_20200902_02_T1_SR_B3.TIF',
 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/0_raw/OPTICAL/LT05_L2SP_227062_20050701_20200902_02_T1\\LT05_L2SP_227062_20050701_20200902_02_T1_SR_B4.TIF',
 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/0_raw/OPTICAL/LT05_L2SP_227062_20050701_20200902_02_T1\\LT05_L2SP_227062_20050701_20200902_02_T1_SR_B5.TIF',
 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/0_raw/OPTICAL/LT05_L2SP_227062_20050701_20200902_02_T1\\LT05_L2SP_227062_20050701_20200902_02_T1_SR_B7.TIF']

In [14]:
# Abre a primeira imagem para obter metadados
with rasterio.open(arquivos_imagens[0]) as src0:
    meta = src0.meta

# Atualiza os metadados para o número de bandas
meta.update(count = len(arquivos_imagens))

# Escreve o arquivo de saída
empilhado_filename = os.path.join(pasta_destino, 'empilhado.tif')
with rasterio.open(empilhado_filename, 'w', **meta) as dst:
    # Escreve cada imagem como uma banda separada
    for id, layer in enumerate(arquivos_imagens, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

print('BANDAS EMPILHADAS!')

BANDAS EMPILHADAS!


In [15]:
with rasterio.open(empilhado_filename) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    reprojetado_filename = os.path.join(pasta_destino, 'reprojetado.tif')
    with rasterio.open(reprojetado_filename, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
print('IMAGEM REPROJETADA')

IMAGEM REPROJETADA


In [17]:
# Recorta a imagem reprojetada usando o vetor
with fiona.open(arquivo_vetor, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(reprojetado_filename) as src:
    out_image, out_transform = mask(src, shapes, crop=True)
    out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Escreve a imagem recortada
nome_pasta = os.path.basename(os.path.normpath(pasta_destino))
recortado_filename = os.path.join(pasta_destino, f'REC_{nome_pasta}.tif')
with rasterio.open(recortado_filename, "w", **out_meta) as dest:
    dest.write(out_image)
print('IMAGEM RECORTADA')

IMAGEM RECORTADA


In [12]:
imagem_sar = 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/1_processed_images/0_SAR/2010/ALOS-H1_1__A-ORBIT__ALPSRP234707120_Cal_ML_DSk_TC_modificado_aligned.TIF'
pasta_destino = 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/1_processed_images/0_SAR/2010/'
arquivo_vetor = 'G:/Meu Drive/INPE/projeto_dissertacao/3_gis/projetos_GIS/PALSAR_2008_SDNLM_amplitude_extention.shp'

In [14]:
# Recorta a imagem reprojetada usando o vetor
with fiona.open(arquivo_vetor, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(imagem_sar) as src:
    out_image, out_transform = mask(src, shapes, crop=True)
    out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Escreve a imagem recortada
nome_pasta = os.path.basename(os.path.normpath(pasta_destino))
recortado_filename = os.path.join(pasta_destino, f'REC2_ALOS-H1_1__A-ORBIT__ALPSRP234707120_Cal_ML_DSk_TC_modificado_aligned.tif')
with rasterio.open(recortado_filename, "w", **out_meta) as dest:
    dest.write(out_image)
print('IMAGEM RECORTADA')

IMAGEM RECORTADA


Reamostragem

In [10]:
import rasterio
from rasterio.enums import Resampling
import os

def reamostrar_imagem(caminho_imagem, fator_reamostragem, caminho_saida):
    """
    Reamostra uma imagem raster, alterando o tamanho do pixel e mantendo os mesmos limites geográficos.

    :param caminho_imagem: Caminho para a imagem de entrada.
    :param fator_reamostragem: Fator pelo qual o tamanho do pixel será alterado. 
                               Por exemplo, 0.5 reduzirá o tamanho do pixel pela metade.
    :param caminho_saida: Caminho para a imagem de saída.
    """
    with rasterio.open(caminho_imagem) as dataset:
        # Calcula a nova resolução (tamanho do pixel)
        nova_resolucao = (dataset.res[0] * fator_reamostragem, dataset.res[1] * fator_reamostragem)

        # Define os novos metadados
        kwargs = dataset.meta.copy()
        kwargs.update({
            'driver': 'GTiff',
            'transform': rasterio.Affine(nova_resolucao[0], dataset.transform.b, dataset.transform.c,
                                         dataset.transform.d, -nova_resolucao[1], dataset.transform.f),
            'width': int(dataset.width / fator_reamostragem),
            'height': int(dataset.height / fator_reamostragem)
        })

        # Reamostra a imagem usando o método 'nearest neighbour'
        with rasterio.open(caminho_saida, 'w', **kwargs) as dst:
            for i in range(1, dataset.count + 1):
                reproject(
                    source=rasterio.band(dataset, i),
                    destination=rasterio.band(dst, i),
                    src_transform=dataset.transform,
                    src_crs=dataset.crs,
                    dst_transform=kwargs['transform'],
                    dst_crs=dataset.crs,
                    resampling=Resampling.nearest
                )


In [11]:
# Exemplo de uso
caminho_imagem = 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/1_processed_images/1_OPTICAL/REC_LT05_L2SP_227062_20100629_20200823_02_T1.TIF'
fator_reamostragem = 0.5  # Para mudar de 30m para 15m, use um fator de 0.5
caminho_saida = 'G:/Meu Drive/INPE/projeto_dissertacao/0_images/1_processed_images/1_OPTICAL/REC_LT05_L2SP_227062_20100629_20200823_02_T1_resampled.TIF'  # Substitua pelo caminho de saída desejado

reamostrar_imagem(caminho_imagem, fator_reamostragem, caminho_saida)